In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
NETWORK = 'ConvNeXtBase'

METHOD = "STD-Median"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.642552,0.956448,1.218982,1.071072,0.804769,0.590774,0.464659,0.846775,0.562021,...,0.833851,0.519748,0.737180,0.702988,0.836509,0.622030,0.469906,1.164363,0.930609,1.039904
1,CHUM-002,0.660845,0.832063,0.990254,1.356161,0.886986,0.541006,0.493334,0.853409,0.589876,...,0.842833,0.515384,0.784886,0.805783,0.888616,0.574868,0.574123,1.075358,1.203229,1.097582
2,CHUM-006,0.592244,0.713517,1.150971,0.985916,0.926790,0.665753,0.581423,0.832329,0.778436,...,0.921583,0.689435,0.651486,0.755239,0.818678,0.583799,0.486945,0.782029,1.345326,1.018169
3,CHUM-007,0.586229,0.674260,1.161925,1.105420,0.901179,0.601261,0.530582,0.986825,0.838722,...,0.914996,0.580232,0.793665,0.725117,0.786759,0.569652,0.536766,0.867290,1.302925,1.083548
4,CHUM-008,0.582757,0.732514,1.117737,1.170768,0.986540,0.643569,0.595419,0.841419,0.785769,...,0.916407,0.636626,0.608093,0.672875,0.800231,0.616226,0.545735,0.758015,1.315464,1.193394


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.642552,0.956448,1.218982,1.071072,0.804769,0.590774,0.464659,0.846775,0.562021,...,0.833851,0.519748,0.737180,0.702988,0.836509,0.622030,0.469906,1.164363,0.930609,1.039904
1,CHUM-002,0.660845,0.832063,0.990254,1.356161,0.886986,0.541006,0.493334,0.853409,0.589876,...,0.842833,0.515384,0.784886,0.805783,0.888616,0.574868,0.574123,1.075358,1.203229,1.097582
2,CHUM-006,0.592244,0.713517,1.150971,0.985916,0.926790,0.665753,0.581423,0.832329,0.778436,...,0.921583,0.689435,0.651486,0.755239,0.818678,0.583799,0.486945,0.782029,1.345326,1.018169
3,CHUM-007,0.586229,0.674260,1.161925,1.105420,0.901179,0.601261,0.530582,0.986825,0.838722,...,0.914996,0.580232,0.793665,0.725117,0.786759,0.569652,0.536766,0.867290,1.302925,1.083548
4,CHUM-008,0.582757,0.732514,1.117737,1.170768,0.986540,0.643569,0.595419,0.841419,0.785769,...,0.916407,0.636626,0.608093,0.672875,0.800231,0.616226,0.545735,0.758015,1.315464,1.193394


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,0.642552,0.956448,1.218982,1.071072,0.804769,0.590774,0.464659,0.846775,0.562021,...,0.737180,0.702988,0.836509,0.622030,0.469906,1.164363,0.930609,1.039904,0,1704
1,CHUM-002,0.660845,0.832063,0.990254,1.356161,0.886986,0.541006,0.493334,0.853409,0.589876,...,0.784886,0.805783,0.888616,0.574868,0.574123,1.075358,1.203229,1.097582,1,439
2,CHUM-006,0.592244,0.713517,1.150971,0.985916,0.926790,0.665753,0.581423,0.832329,0.778436,...,0.651486,0.755239,0.818678,0.583799,0.486945,0.782029,1.345326,1.018169,0,1186
3,CHUM-007,0.586229,0.674260,1.161925,1.105420,0.901179,0.601261,0.530582,0.986825,0.838722,...,0.793665,0.725117,0.786759,0.569652,0.536766,0.867290,1.302925,1.083548,0,1702
4,CHUM-008,0.582757,0.732514,1.117737,1.170768,0.986540,0.643569,0.595419,0.841419,0.785769,...,0.608093,0.672875,0.800231,0.616226,0.545735,0.758015,1.315464,1.193394,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)